Importing libraries

In [42]:
import os
import glob
import pandas as pd 
import csv
import collections
import pprint

We display the preprocessed .csv files

In [2]:
data_directory = '../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/'
raw_files = os.listdir(data_directory)

raw_files

['environmental.csv',
 'luxmeter.csv',
 'luxmeter2018.csv',
 'luxmeter2019.csv',
 'powerelectricity.csv',
 'presence.csv',
 'presence2018.csv',
 'presence2019.csv',
 'reedsensor.csv']

<h3>Now we will verify the integrity of the data of each .csv file 
(Excluding presence.csv and luxmeter.csv because we will consider only files divided by years)</h3> 
<h4 style="color:red"> Environmental.csv </h4>

In [15]:
df = pd.read_csv("../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/environmental.csv")
df.info()
singleDataNull = sum(df.isnull().values.ravel())
singleRowContainingNull = sum([True for idx,row in df.iterrows() if any(row.isnull())])

print('We have',singleDataNull,'null values in',singleRowContainingNull,'rows')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48173 entries, 0 to 48172
Data columns (total 17 columns):
idsensor                     48173 non-null object
sensortype                   48173 non-null object
srtDate                      48173 non-null object
absolute_humidity            48173 non-null float64
back_front                   48173 non-null float64
battery_level                48173 non-null float64
device_temperature           48044 non-null float64
dew_point                    48044 non-null float64
environmental_temperature    48044 non-null float64
movement_level               48044 non-null float64
orientation                  48044 non-null float64
position                     48044 non-null float64
posture_tilt                 48044 non-null float64
pressure                     48044 non-null float64
regular                      48044 non-null float64
relative_humidity            48044 non-null float64
sequence_number              48044 non-null float64
dtypes: fl

Due to the lack of device data in 129 records, I decided to exclude these rows.
Since in total only 129 lines out of 48173 contain a null value, we can proceed with the elimination of the latter
then saving the .csv file

In [23]:
try:
    df.dropna(inplace=True)
    df.to_csv('../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Cleaned data/environmental.csv')
except OSError:
    print ("Creation of the file failed")
else:
    print("environmental.csv successfully saved!")




environmental.csv successfully saved!


<h4 style="color:red"> Powerelectricity.csv </h4>

In [25]:
df = pd.read_csv("../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/powerelectricity.csv")
df.info()
singleDataNull = sum(df.isnull().values.ravel())
singleRowContainingNull = sum([True for idx,row in df.iterrows() if any(row.isnull())])

print('We have',singleDataNull,'null values in',singleRowContainingNull,'rows')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166070 entries, 0 to 166069
Data columns (total 5 columns):
idsensor          166070 non-null object
sensortype        166070 non-null object
srtDate           166070 non-null object
electric_power    166070 non-null float64
rms_current       166070 non-null float64
dtypes: float64(2), object(3)
memory usage: 6.3+ MB
We have 0 null values in 0 rows


No null value seems to be present here. We can save the file without any operation

In [26]:
try:
    df.to_csv('../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Cleaned data/powerelectricity.csv')
except OSError:
    print ("Creation of the file failed")
else:
    print("powerelectricity.csv successfully saved!")


powerelectricity.csv successfully saved!


<h4 style="color:red"> Reedsensor.csv </h4>

In [44]:
df = pd.read_csv("../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/reedsensor.csv")
df.info()
singleDataNull = sum(df.isnull().values.ravel())
singleRowContainingNull = sum([True for idx,row in df.iterrows() if any(row.isnull())])

print('We have',singleDataNull,'null values in',singleRowContainingNull,'rows')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156958 entries, 0 to 156957
Data columns (total 7 columns):
idsensor              156958 non-null object
sensortype            156958 non-null object
srtDate               156958 non-null object
battery_level         156958 non-null float64
device_temperature    156958 non-null float64
sequence_number       156958 non-null float64
status                156919 non-null float64
dtypes: float64(4), object(3)
memory usage: 8.4+ MB
We have 39 null values in 39 rows


There seems to be a problem in 39 "status" column values, we see in detail:

In [48]:
df_null_status = df[df['status'].isnull()]
df_null_status.head()

,idsensor,sensortype,srtDate,battery_level,device_temperature,sequence_number,status
14,jzp://edv#0126.0000,ReedSensor,2018-03-01T01:11:11.397+01:00,3.55,13.0,0.0,NaN
4539,jzp://edv#0126.0000,ReedSensor,2018-03-03T13:10:29.557+01:00,3.55,12.0,0.0,NaN
8709,jzp://edv#0126.0000,ReedSensor,2018-03-06T01:11:40.630+01:00,3.55,12.0,0.0,NaN
13447,jzp://edv#0126.0000,ReedSensor,2018-03-08T13:10:47.485+01:00,3.55,14.0,0.0,NaN
18173,jzp://edv#0126.0000,ReedSensor,2018-03-11T01:11:45.825+01:00,3.55,13.0,0.0,NaN


Now let's look at the behavior of the "status" value when it is not null

In [51]:
df.groupby('status').agg(['count']).stack()

,,idsensor,sensortype,srtDate,battery_level,device_temperature,sequence_number
status,,,,,,,
0.0,count,152979,152979,152979,152979,152979,152979
1.0,count,3940,3940,3940,3940,3940,3940


In addition to the 39 null values, there are 3940 "status" values ​​set to 1, so this value should indicate something specific and therefore we cannot set the 39 null values ​​to 0 to solve the problem. 
Then we will save the new file with the null values ​​setted with a marker that we will indicate with "*"

In [56]:
df.fillna("*",inplace=True)
df.info()
df.groupby('status').agg(['count']).stack()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156958 entries, 0 to 156957
Data columns (total 7 columns):
idsensor              156958 non-null object
sensortype            156958 non-null object
srtDate               156958 non-null object
battery_level         156958 non-null float64
device_temperature    156958 non-null float64
sequence_number       156958 non-null float64
status                156958 non-null object
dtypes: float64(3), object(4)
memory usage: 8.4+ MB


,,idsensor,sensortype,srtDate,battery_level,device_temperature,sequence_number
status,,,,,,,
0.0,count,152979,152979,152979,152979,152979,152979
1.0,count,3940,3940,3940,3940,3940,3940
*,count,39,39,39,39,39,39


Now let's save the file cleaned

In [57]:
try:
    df.to_csv('../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Cleaned data/reedsensor.csv')
except OSError:
    print ("Creation of the file failed")
else:
    print("reedsensor.csv successfully saved!")

powerelectricity.csv successfully saved!


<h4 style="color:red"> luxmeter2018.csv / luxmeter2019.csv </h4> 

In [3]:
df2018 = pd.read_csv("../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/luxmeter2018.csv")
df2018.info()

singleDataNull2018 = sum(df2018.isnull().values.ravel())
singleRowContainingNull2018 = sum([True for idx,row in df2018.iterrows() if any(row.isnull())])

print('In the file luxmeter2018.csv We have',singleDataNull2018,'null values in',singleRowContainingNull2018,'rows')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192271 entries, 0 to 192270
Data columns (total 9 columns):
idsensor              192271 non-null object
sensortype            192271 non-null object
srtDate               192271 non-null object
adc_channel_00        192271 non-null float64
adc_channel_01        192271 non-null float64
battery_level         192271 non-null float64
device_temperature    192131 non-null float64
illuminance           192131 non-null float64
sequence_number       192131 non-null float64
dtypes: float64(6), object(3)
memory usage: 13.2+ MB
In the file luxmeter2018.csv We have 420 null values in 140 rows


Ok we have 420 null values in 140 rows... let's see an example on table:

In [5]:
df_null = df2018[df2018['device_temperature'].isnull()]
df_null.head()

,idsensor,sensortype,srtDate,adc_channel_00,adc_channel_01,battery_level,device_temperature,illuminance,sequence_number
688,jzp://edv#0303.0000,LuxMeter,2018-03-01T06:45:02.750+01:00,3.25,23.0,1.0,NaN,NaN,NaN
2129,jzp://edv#0303.0000,LuxMeter,2018-03-01T18:47:20.568+01:00,3.25,24.0,1.0,NaN,NaN,NaN
3321,jzp://edv#0303.0000,LuxMeter,2018-03-02T04:48:00.601+01:00,3.25,24.0,1.0,NaN,NaN,NaN
4760,jzp://edv#0303.0000,LuxMeter,2018-03-02T16:50:18.572+01:00,3.25,24.0,1.0,NaN,NaN,NaN
5272,jzp://edv#0303.0000,LuxMeter,2018-03-02T21:08:57.201+01:00,3.25,24.0,1.0,NaN,NaN,NaN


Here the most convenient action is to eliminate these rows because the number of rows we're going to delete correspond to the 0.07% of the total (140 on 192270)

In [7]:
df2018.dropna(inplace=True)
df2018.info()
try:
    df2018.to_csv('../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Cleaned data/luxmeter2018.csv')
except OSError:
    print ("Creation of the file failed")
else:
    print("luxmeter2018.csv successfully saved!")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192131 entries, 0 to 192270
Data columns (total 9 columns):
idsensor              192131 non-null object
sensortype            192131 non-null object
srtDate               192131 non-null object
adc_channel_00        192131 non-null float64
adc_channel_01        192131 non-null float64
battery_level         192131 non-null float64
device_temperature    192131 non-null float64
illuminance           192131 non-null float64
sequence_number       192131 non-null float64
dtypes: float64(6), object(3)
memory usage: 14.7+ MB
luxmeter2018.csv successfully saved!


As for the 2018 data, we have the same problem for the 2019 data so I will proceed in the same way

In [8]:
df2019 = pd.read_csv("../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/luxmeter2019.csv")
df2019.info()

singleDataNull2019 = sum(df2019.isnull().values.ravel())
singleRowContainingNull2019 = sum([True for idx,row in df2019.iterrows() if any(row.isnull())])

print('In the file luxmeter2019.csv We have',singleDataNull2019,'null values in',singleRowContainingNull2019,'rows')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255002 entries, 0 to 255001
Data columns (total 9 columns):
idsensor              255002 non-null object
sensortype            255002 non-null object
srtDate               255002 non-null object
adc_channel_00        255002 non-null float64
adc_channel_01        255002 non-null float64
battery_level         255002 non-null float64
device_temperature    254819 non-null float64
illuminance           254819 non-null float64
sequence_number       254819 non-null float64
dtypes: float64(6), object(3)
memory usage: 17.5+ MB
In the file luxmeter2019.csv We have 549 null values in 183 rows


In [9]:
df_null = df2019[df2019['device_temperature'].isnull()]
df_null.head()


,idsensor,sensortype,srtDate,adc_channel_00,adc_channel_01,battery_level,device_temperature,illuminance,sequence_number
1361,jzp://edv#0303.0000,LuxMeter,2019-02-01T12:29:43.561+01:00,3.25,26.0,1.0,NaN,NaN,NaN
2798,jzp://edv#0303.0000,LuxMeter,2019-02-02T00:32:02.500+01:00,3.25,26.0,1.0,NaN,NaN,NaN
4235,jzp://edv#0303.0000,LuxMeter,2019-02-02T12:34:21.083+01:00,3.25,26.0,1.0,NaN,NaN,NaN
5675,jzp://edv#0303.0000,LuxMeter,2019-02-03T00:36:39.443+01:00,3.25,26.0,1.0,NaN,NaN,NaN
7116,jzp://edv#0303.0000,LuxMeter,2019-02-03T12:38:58.373+01:00,3.25,26.0,1.0,NaN,NaN,NaN


In [10]:
df2019.dropna(inplace=True)
df2019.info()
try:
    df2019.to_csv('../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Cleaned data/luxmeter2019.csv')
except OSError:
    print ("Creation of the file failed")
else:
    print("luxmeter2019.csv successfully saved!")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254819 entries, 0 to 255001
Data columns (total 9 columns):
idsensor              254819 non-null object
sensortype            254819 non-null object
srtDate               254819 non-null object
adc_channel_00        254819 non-null float64
adc_channel_01        254819 non-null float64
battery_level         254819 non-null float64
device_temperature    254819 non-null float64
illuminance           254819 non-null float64
sequence_number       254819 non-null float64
dtypes: float64(6), object(3)
memory usage: 19.4+ MB
luxmeter2019.csv successfully saved!


<h4 style="color:red"> presence2018.csv / presence2019.csv </h4> 

In [11]:
dfp2018 = pd.read_csv("../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/presence2018.csv")
dfp2018.info()

singleDataNull2018 = sum(dfp2018.isnull().values.ravel())
singleRowContainingNull2018 = sum([True for idx,row in dfp2018.iterrows() if any(row.isnull())])

print('In the file presence2018.csv We have',singleDataNull2018,'null values in',singleRowContainingNull2018,'rows')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440499 entries, 0 to 440498
Data columns (total 7 columns):
idsensor              440499 non-null object
sensortype            440499 non-null object
srtDate               440499 non-null object
battery_level         440499 non-null float64
device_temperature    440499 non-null float64
presence              440499 non-null float64
sequence_number       440242 non-null float64
dtypes: float64(4), object(3)
memory usage: 23.5+ MB
In the file presence2018.csv We have 257 null values in 257 rows


Even here we have a problem similar to the reedsensor in the column "sequence_number", because we have only 1 value missing. let's see in detail:

In [12]:
df_null = dfp2018[dfp2018['sequence_number'].isnull()]
df_null.head()


,idsensor,sensortype,srtDate,battery_level,device_temperature,presence,sequence_number
1544,jzp://edv#0321.0000,Presence,2018-03-03T04:06:09.438+01:00,3.45,4.0,0.0,NaN
3415,jzp://edv#0321.0000,Presence,2018-03-05T16:06:06.208+01:00,3.45,5.0,0.0,NaN
5365,jzp://edv#0321.0000,Presence,2018-03-08T04:06:00.148+01:00,3.45,5.0,0.0,NaN
7987,jzp://edv#0321.0000,Presence,2018-03-10T16:05:58.243+01:00,3.45,9.0,0.0,NaN
9797,jzp://edv#0321.0000,Presence,2018-03-13T04:05:52.973+01:00,3.45,7.0,0.0,NaN


Ok now let's see if the value of column 12 can be replaced with a "*" as we did in the reedsensor case

In [14]:
dfp2018.groupby('sequence_number').agg(['count']).stack()

,,idsensor,sensortype,srtDate,battery_level,device_temperature,presence
sequence_number,,,,,,,
0.0,count,1632,1632,1632,1632,1632,1632
1.0,count,1882,1882,1882,1882,1882,1882
2.0,count,1879,1879,1879,1879,1879,1879
3.0,count,1883,1883,1883,1883,1883,1883
4.0,count,1868,1868,1868,1868,1868,1868
...,...,...,...,...,...,...,...
251.0,count,1614,1614,1614,1614,1614,1614
252.0,count,1616,1616,1616,1616,1616,1616
253.0,count,1624,1624,1624,1624,1624,1624


Ok we have a value that goes between 1 and 255, let's see a specific case where we have this null value


In [16]:
dfp2018.iloc[1541:1548]

,idsensor,sensortype,srtDate,battery_level,device_temperature,presence,sequence_number
1541,jzp://edv#0321.0000,Presence,2018-03-03T04:00:02.593+01:00,3.45,4.0,0.0,19.0
1542,jzp://edv#0321.0000,Presence,2018-03-03T04:02:02.820+01:00,3.45,4.0,0.0,20.0
1543,jzp://edv#0321.0000,Presence,2018-03-03T04:04:02.822+01:00,3.45,4.0,0.0,21.0
1544,jzp://edv#0321.0000,Presence,2018-03-03T04:06:09.438+01:00,3.45,4.0,0.0,NaN
1545,jzp://edv#0321.0000,Presence,2018-03-03T04:08:09.485+01:00,3.45,4.0,0.0,1.0
1546,jzp://edv#0321.0000,Presence,2018-03-03T04:10:08.521+01:00,3.45,4.0,0.0,2.0
1547,jzp://edv#0321.0000,Presence,2018-03-03T04:12:08.333+01:00,3.45,4.0,0.0,3.0


In [17]:
dfp2018.iloc[3412:3419]

,idsensor,sensortype,srtDate,battery_level,device_temperature,presence,sequence_number
3412,jzp://edv#0321.0000,Presence,2018-03-05T15:59:57.167+01:00,3.45,4.0,0.0,77.0
3413,jzp://edv#0321.0000,Presence,2018-03-05T16:01:57.128+01:00,3.45,4.0,0.0,78.0
3414,jzp://edv#0321.0000,Presence,2018-03-05T16:03:57.060+01:00,3.45,4.0,0.0,79.0
3415,jzp://edv#0321.0000,Presence,2018-03-05T16:06:06.208+01:00,3.45,5.0,0.0,NaN
3416,jzp://edv#0321.0000,Presence,2018-03-05T16:08:02.979+01:00,3.45,5.0,0.0,1.0
3417,jzp://edv#0321.0000,Presence,2018-03-05T16:10:02.792+01:00,3.45,5.0,0.0,2.0
3418,jzp://edv#0321.0000,Presence,2018-03-05T16:12:03.585+01:00,3.45,5.0,0.0,3.0


In [18]:
dfp2018.iloc[5362:5369]

,idsensor,sensortype,srtDate,battery_level,device_temperature,presence,sequence_number
5362,jzp://edv#0321.0000,Presence,2018-03-08T03:59:53.128+01:00,3.45,6.0,0.0,157.0
5363,jzp://edv#0321.0000,Presence,2018-03-08T04:01:53.333+01:00,3.45,6.0,0.0,158.0
5364,jzp://edv#0321.0000,Presence,2018-03-08T04:03:53.172+01:00,3.45,6.0,0.0,159.0
5365,jzp://edv#0321.0000,Presence,2018-03-08T04:06:00.148+01:00,3.45,5.0,0.0,NaN
5366,jzp://edv#0321.0000,Presence,2018-03-08T04:07:58.498+01:00,3.45,5.0,0.0,1.0
5367,jzp://edv#0321.0000,Presence,2018-03-08T04:09:58.184+01:00,3.45,5.0,0.0,2.0
5368,jzp://edv#0321.0000,Presence,2018-03-08T04:11:59.033+01:00,3.45,5.0,0.0,3.0


Ok it seems that this null value indicates 0 because every row containing the null value resets the count. So instead of the null value we will insert the 0 and we will save the new file

In [19]:
dfp2018.fillna("0",inplace=True)
dfp2018.info()

try:
    dfp2018.to_csv('../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Cleaned data/presence2018.csv')
except OSError:
    print ("Creation of the file failed")
else:
    print("presencce2018.csv successfully saved!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440499 entries, 0 to 440498
Data columns (total 7 columns):
idsensor              440499 non-null object
sensortype            440499 non-null object
srtDate               440499 non-null object
battery_level         440499 non-null float64
device_temperature    440499 non-null float64
presence              440499 non-null float64
sequence_number       440499 non-null object
dtypes: float64(3), object(4)
memory usage: 23.5+ MB
presencce2018.csv successfully saved!


Now let's analyze the last file, presence2019.csv

In [30]:
dfp2019 = pd.read_csv("../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/presence2019.csv")
dfp2019.info()

singleDataNull2019 = sum(dfp2019.isnull().values.ravel())
singleRowContainingNull2019 = sum([True for idx,row in dfp2019.iterrows() if any(row.isnull())])

print('In the file presence2019.csv We have',singleDataNull2019,'null values in',singleRowContainingNull2019,'rows')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789879 entries, 0 to 789878
Data columns (total 7 columns):
idsensor              789879 non-null object
sensortype            789879 non-null object
srtDate               789879 non-null object
battery_level         789879 non-null float64
device_temperature    789879 non-null float64
presence              789879 non-null float64
sequence_number       789435 non-null float64
dtypes: float64(4), object(3)
memory usage: 42.2+ MB
In the file presence2019.csv We have 444 null values in 444 rows



Ok here too the same problem of the previous year. We verify that even here we have the "countdown" on column sequence_number.

In [23]:
df_null = dfp2019[dfp2019['sequence_number'].isnull()]
df_null.head()

,idsensor,sensortype,srtDate,battery_level,device_temperature,presence,sequence_number
1469,jzp://edv#0325.0000,Presence,2019-02-01T12:25:28.157+01:00,3.45,3.0,0.0,NaN
3047,jzp://edv#0325.0000,Presence,2019-02-02T00:25:38.570+01:00,3.45,-1.0,0.0,NaN
4481,jzp://edv#0325.0000,Presence,2019-02-02T12:25:48.075+01:00,3.45,-1.0,0.0,NaN
5921,jzp://edv#0325.0000,Presence,2019-02-03T00:25:57.812+01:00,3.45,-1.0,0.0,NaN
7359,jzp://edv#0325.0000,Presence,2019-02-03T12:26:08.170+01:00,3.45,-1.0,0.0,NaN


In [25]:
dfp2019.iloc[1466:1472]

,idsensor,sensortype,srtDate,battery_level,device_temperature,presence,sequence_number
1466,jzp://edv#0325.0000,Presence,2019-02-01T12:23:51.569+01:00,3.45,3.0,0.0,22.0
1467,jzp://edv#0325.0000,Presence,2019-02-01T12:24:21.751+01:00,3.45,3.0,0.0,23.0
1468,jzp://edv#0325.0000,Presence,2019-02-01T12:24:51.557+01:00,3.45,3.0,0.0,24.0
1469,jzp://edv#0325.0000,Presence,2019-02-01T12:25:28.157+01:00,3.45,3.0,0.0,NaN
1470,jzp://edv#0325.0000,Presence,2019-02-01T12:25:56.947+01:00,3.45,3.0,0.0,1.0
1471,jzp://edv#0325.0000,Presence,2019-02-01T12:26:27.273+01:00,3.45,3.0,0.0,2.0


In [27]:
dfp2019.iloc[3044:3050]

,idsensor,sensortype,srtDate,battery_level,device_temperature,presence,sequence_number
3044,jzp://edv#0325.0000,Presence,2019-02-02T00:24:01.659+01:00,3.45,-1.0,0.0,45.0
3045,jzp://edv#0325.0000,Presence,2019-02-02T00:24:32.086+01:00,3.45,-1.0,0.0,46.0
3046,jzp://edv#0325.0000,Presence,2019-02-02T00:25:01.489+01:00,3.45,-1.0,0.0,47.0
3047,jzp://edv#0325.0000,Presence,2019-02-02T00:25:38.570+01:00,3.45,-1.0,0.0,NaN
3048,jzp://edv#0325.0000,Presence,2019-02-02T00:26:07.602+01:00,3.45,-1.0,0.0,1.0
3049,jzp://edv#0325.0000,Presence,2019-02-02T00:26:37.201+01:00,3.45,-1.0,0.0,2.0



Even here we have the same problem, so we will solve it as in the 2018 data.

In [28]:
dfp2019.fillna("0",inplace=True)
dfp2019.info()

try:
    dfp2019.to_csv('../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Cleaned data/presence2019.csv')
except OSError:
    print ("Creation of the file failed")
else:
    print("presence2019.csv successfully saved!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789879 entries, 0 to 789878
Data columns (total 7 columns):
idsensor              789879 non-null object
sensortype            789879 non-null object
srtDate               789879 non-null object
battery_level         789879 non-null float64
device_temperature    789879 non-null float64
presence              789879 non-null float64
sequence_number       789879 non-null object
dtypes: float64(3), object(4)
memory usage: 42.2+ MB
presence2019.csv successfully saved!



With this last step we have concluded the data cleaning. However, analyzing the presence sensor data separately, I noticed that the sensorID changed from 2018 to 2019 so these two files need more specific controls

In [37]:
cleaned_presence2018=pd.read_csv("../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/presence2018.csv")
cleaned_presence2019=pd.read_csv("../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/presence2019.csv")

cleaned_presence2018idsensor = cleaned_presence2018['idsensor'].unique()
print('This is the array of different sensor types for the 2018 file:',cleaned_presence2018idsensor,)

cleaned_presence2019idsensor = cleaned_presence2019['idsensor'].unique()
print('This is the array of different sensor types for the 2019 file:',cleaned_presence2019idsensor,)


This is the array of different sensor types for the 2018 file: ['jzp://edv#0321.0000' 'jzp://edv#0325.0000' 'jzp://edv#0322.0000'
 'jzp://edv#0320.0000']
This is the array of different sensor types for the 2019 file: ['jzp://edv#0325.0000' 'jzp://edv#0324.0000' 'jzp://edv#0322.0000'
 'jzp://edv#0323.0000' 'jzp://edv#0320.0000']



It may be appropriate to divide the files also based on the type of sensor to which they refer

In [46]:
for sensor in cleaned_presence2019idsensor:
    sensorName2019 = sensor.replace('jzp://edv#', '')
    sensorName2019 = sensorName2019.replace('.0000', '')
    temp_df = dfp2019[dfp2019['idsensor'] == sensor]
    temp_df.to_csv('../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Cleaned data/presence2019-Sensor' + sensorName2019+ '.csv')
    print('file presence2019',sensorName2019,'creato')
    
for sensor in cleaned_presence2018idsensor:
    sensorName2018 = sensor.replace('jzp://edv#', '')
    sensorName2018 = sensorName2018.replace('.0000', '')
    temp_df = dfp2018[dfp2018['idsensor'] == sensor]
    temp_df.to_csv('../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Cleaned data/presence2018-Sensor' + sensorName2018 + '.csv')
    print('file presence2018',sensorName2018,'creato')

file presence2019 0325 creato
file presence2019 0324 creato
file presence2019 0322 creato
file presence2019 0323 creato
file presence2019 0320 creato
file presence2018 0321 creato
file presence2018 0325 creato
file presence2018 0322 creato
file presence2018 0320 creato
